In [ ]:
import keras
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize

def encode(series):
    return pd.get_dummies(series.astype(str))

def mean_absolute_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred)))

class traininghist(keras.callbacks.Callback):
    def __init__(self, test_data):
        self.test_data = test_data
        self.trainingloss = []
        self.trainingmape = []
        
    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, mape = self.model.evaluate(x, y, verbose=0)
        self.trainingloss.append(loss)
        self.trainingmape.append(mape)
        print('Training loss: {}, mape: {}\n'.format(loss, mape))

In [ ]:
#Import training data

features1 = np.loadtxt('vec_mat_features_swellex_0.01train_109hz2.csv',delimiter=",")
#features2 = np.loadtxt('vec_mat_features_swellex_0.01trainbb_163hz.csv',delimiter=",")
#features3 = np.loadtxt('vec_mat_features_swellex_0.01trainbb_232hz.csv',delimiter=",")
#features4 = np.loadtxt('vec_mat_features_swellex_0.01trainbb_385hz.csv',delimiter=",")
labels_unstand = np.loadtxt('vec_mat_rlabels_swellex_0.01trainbb.csv',delimiter=",")
labels_t = []

features_mat = np.array([[features1]])#,[features2],[features3],[features4]])
X_train = np.zeros((features1.shape[0],21,21,2))

for f in range(features_mat.shape[0]):
    print(f)
    features = features_mat[f,0,:,:]
    real = features[:,0::2]
    imag = features[:,1::2]
    
    for k in range(features.shape[0]):
        count = 0
        for i in range(21):
            for j in range(i,21):
                X_train[k,i,j,2*f] = real[k,count]
                X_train[k,i,j,2*f+1] = imag[k,count]
                
                if i!=j:
                    X_train[k,j,i,2*f] = X_train[k,i,j,2*f]
                    X_train[k,j,i,2*f+1] = -X_train[k,i,j,2*f+1]
                
                count = count + 1

#X_train[k,:,:,0] = X_train[k,:,:,0]/np.amax(np.abs(X_train[k,:,:,0]))
#X_train[k,:,:,1] = X_train[k,:,:,1]/np.amax(np.abs(X_train[k,:,:,1]))

labels_unstand = labels_unstand.ravel()
#y_train,mu,sigma_labels = std_y(labels_unstand)

y_train = labels_unstand
y_train = y_train


In [ ]:
#Import test data

features_test1 = np.loadtxt('vec_mat_features_swellex_109hz2_test6.csv',delimiter=",")
#features_test2 = np.loadtxt('vec_mat_features_swellex_test2_163hz.csv',delimiter=",")
#features_test3 = np.loadtxt('vec_mat_features_swellex_test2_232hz.csv',delimiter=",")
#features_test4 = np.loadtxt('vec_mat_features_swellex_test2_385hz.csv',delimiter=",")
temp_ytest = np.loadtxt('vec_mat_rlabels_swellex_109hz2_test.csv',delimiter=",")
y_test= []

featurestest_mat = np.array([[features_test1]])#,[features_test2],[features_test3],[features_test4]])
X_test = np.zeros((features_test1.shape[0],21,21,2))

for f in range(featurestest_mat.shape[0]):
    print(f)
    features_test = featurestest_mat[f,0,:,:]
    real_test = features_test[:,0::2]
    imag_test = features_test[:,1::2]

    for k in range(features_test.shape[0]):
        count = 0
        for i in range(21):
            for j in range(i,21):
                X_test[k,i,j,2*f] = real_test[k,count]
                X_test[k,i,j,2*f+1] = imag_test[k,count]
            
                if i!=j:
                    X_test[k,j,i,2*f] = X_test[k,i,j,2*f]
                    X_test[k,j,i,2*f+1] = -X_test[k,i,j,2*f+1]
            
                count = count + 1

#X_test[k,:,:,0] = X_test[k,:,:,0]/np.amax(np.abs(X_test[k,:,:,0]))
#X_test[k,:,:,1] = X_test[k,:,:,1]/np.amax(np.abs(X_test[k,:,:,1]))

temp_ytest = temp_ytest.ravel()
#y_test = (temp_ytest - mu)/sigma_labels
y_test = temp_ytest
y_test = y_test


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
#Training

index=np.arange(X_train.shape[0])
np.random.shuffle(index)

X_train=X_train[index,:,:,:]
y_train=y_train[index]

drate = 0.25
n_node = 256
batch_size = 128
loss='mean_squared_error'

model = keras.Sequential([
                          keras.layers.Conv2D(input_shape=(21,21,2), filters=16, kernel_size=3,padding='same',activation='selu',strides=(2,2)),
                          keras.layers.BatchNormalization(),
                          keras.layers.Dropout(0.5),
                          keras.layers.Conv2D(filters=128, kernel_size=5,padding='same',activation='selu',strides=(2,2)),
                          keras.layers.BatchNormalization(),
                          keras.layers.Dropout(0.5),
                          keras.layers.Conv2D(filters=256, kernel_size=5,padding='same',activation='selu',strides=(2,2)),
                          keras.layers.BatchNormalization(),
                          keras.layers.Dropout(0.5),
                          #keras.layers.Conv2D(filters=32, kernel_size=3,padding='same',activation='selu',strides=(2,2)),
                          #keras.layers.BatchNormalization(),
                          #keras.layers.Dropout(0.5),
                          #keras.layers.Conv2D(filters=128, kernel_size=3,padding='same',activation='selu',strides=(2,2)),
                          #keras.layers.BatchNormalization(),
                          keras.layers.Flatten(),
                          keras.layers.Dense(units=n_node, activation='sigmoid'),
                          keras.layers.Dropout(drate),
                          keras.layers.Dense(units=1, activation='linear')])

lr = 0.0001
optimizer = keras.optimizers.Adam(lr)

model.compile(optimizer,loss, metrics=['mean_absolute_error'])

filepath = "temp_cnnr.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, mode='auto',period=1)
reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=75, mode='auto')
early = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=125, mode='auto',restore_best_weights=True)
traininghistory = traininghist((X_train,y_train))
callbacks_list = [checkpoint,reduce,traininghistory,early]

infdb = model.fit(X_train, y_train, batch_size,verbose = True, epochs=5000, validation_split=0.2, shuffle=True,callbacks=callbacks_list)

#Testing

test_loss, test_acc = model.evaluate(X_test,y_test)
train_loss, train_acc = model.evaluate(X_train,y_train)

y_train = y_train
y_test = y_test

predictions = model.predict(X_test)
predictions = predictions
diff = abs((np.transpose(predictions))-(y_test))
error = diff[diff>1]
percent_correct = (len(y_test)-len(error))/len(y_test)

predictions_train = model.predict(X_train)
predictions_train = predictions_train
diff = abs((np.transpose(predictions_train))-(y_train))
error = diff[diff>1]
percent_correct_train = (len(y_train)-len(error))/len(y_train)

print('Training mae:', train_acc)
print('Test mae:', test_acc)
print('Training percent within 1km:',percent_correct_train*100)
print('Testing percent within 1km:', percent_correct*100)

In [ ]:
epoch = np.arange(1,len(infdb.history['loss'])+1)
fig1 = plt.figure()
plt.subplot(1,2,1)
plt.suptitle('CNN-r Training Progression')
plt.plot(epoch, infdb.history['loss'], label = 'Training')
plt.plot(epoch, infdb.history['val_loss'],label = 'Testing')
plt.legend()
plt.grid()
plt.xlabel('Epoch Number')
plt.ylabel('MSE Loss')
plt.axis([1, 1000, 0, 8])
plt.subplot(1,2,2)
plt.plot(epoch, np.array(infdb.history['mean_absolute_error']),label = 'Training')
plt.plot(epoch, np.array(infdb.history['val_mean_absolute_error']),label = 'Testing')
plt.legend()
plt.xlabel('Epoch Number')
plt.ylabel('MAE')
plt.grid()
plt.axis([1, 1000, 0, 3])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
fig1.savefig('cnnr_training.png')

In [ ]:
#Testing

test_loss, test_acc = model.evaluate(X_test,y_test)

predictions = model.predict(X_test)
predictions = predictions
diff = abs((np.transpose(predictions))-(y_test))
error = diff[diff>1]
percent_correct = (len(y_test)-len(error))/len(y_test)

print('Test mae:', test_acc)
print('Testing percent within 1km:', percent_correct*100)

In [ ]:
inds = y_test.argsort()
sorted_y_test = y_test[inds]

sorted_y_pred = np.squeeze(predictions)[inds]

#Plotting
colors = sorted_y_pred-sorted_y_test
norm = Normalize()
norm.autoscale(colors)
colormap = cm.rainbow
fig1 = plt.figure(figsize = (10,8))
plt.scatter(sorted_y_test,sorted_y_pred,marker='.',c='k',alpha=0.75)
plt.plot(sorted_y_test,sorted_y_test,'r',linewidth = 2)
plt.quiver(sorted_y_test, sorted_y_test, np.zeros(len(sorted_y_test)), sorted_y_pred-sorted_y_test,color=colormap(norm(colors)),angles='xy', scale_units='xy', scale=1.,width=0.002)
plt.axis([0, 10, 0, 10],fontsize=20)
plt.ylabel('Predicted Distance (km)',fontsize=20)
plt.xlabel('Actual Distance (km)',fontsize=20)
plt.title('CNN Regression Test Predictions (SNR = INF)',fontsize=20)
plt.grid()

In [ ]:
#Import test data

features_test1 = np.loadtxt('vec_mat_features_swellex_shallow_109hz.csv',delimiter=",")
#features_test2 = np.loadtxt('vec_mat_features_swellex_test2_163hz.csv',delimiter=",")
#features_test3 = np.loadtxt('vec_mat_features_swellex_test2_232hz.csv',delimiter=",")
#features_test4 = np.loadtxt('vec_mat_features_swellex_test2_385hz.csv',delimiter=",")
#temp_ytest = np.loadtxt('vec_mat_rlabels_swellex_test2.csv',delimiter=",")
#y_test= []

featurestest_mat = np.array([[features_test1]])#,[features_test2],[features_test3],[features_test4]])
X_test1 = np.zeros((features_test1.shape[0],21,21,2))

for f in range(featurestest_mat.shape[0]):
    print(f)
    features_test11 = featurestest_mat[f,0,:,:]
    real_test = features_test11[:,0::2]
    imag_test = features_test11[:,1::2]

    for k in range(features_test11.shape[0]):
        count = 0
        for i in range(21):
            for j in range(i,21):
                X_test1[k,i,j,2*f] = real_test[k,count]
                X_test1[k,i,j,2*f+1] = imag_test[k,count]
            
                if i!=j:
                    X_test1[k,j,i,2*f] = X_test1[k,i,j,2*f]
                    X_test1[k,j,i,2*f+1] = -X_test1[k,i,j,2*f+1]
            
                count = count + 1

#X_test[k,:,:,0] = X_test[k,:,:,0]/np.amax(np.abs(X_test[k,:,:,0]))
#X_test[k,:,:,1] = X_test[k,:,:,1]/np.amax(np.abs(X_test[k,:,:,1]))

#temp_ytest = temp_ytest.ravel()
#y_test = (temp_ytest - mu)/sigma_labels
#y_test = temp_ytest
#y_test = y_test+5


#print(X_train.shape)
#print(y_train.shape)
print(X_test1.shape)
#print(y_test.shape)

In [ ]:
dist_actual = [8.604,
8.459,
8.324,
8.175,
8.039,
7.888,
7.735,
7.582,
7.449,
7.297,
7.146,
6.995,
6.844,
6.526,
6.375,
6.225,
6.056,
5.904,
5.752,
5.6,
5.467,
5.316,
5.164,
5.033,
4.883,
4.734,
4.585,
4.473,
4.324,
4.194,
4.046,
3.897,
3.767,
3.618,
3.47,
3.321,
3.191,
3.044,
2.915,
2.77,
2.643,
2.5,
2.376,
2.236,
2.113,
1.977,
1.84,
1.706,
1.591,
1.464,
1.334,
1.232,
1.122,
1.022,
0.938,
0.867,
0.812,
0.785,
0.774,
0.785,
0.85,
0.922,
1.012,
1.096,
1.219,
1.337,
1.471,
1.609,
1.736,
1.869,
2.014,
2.151,
2.299,
2.439,
2.589]

In [ ]:
np.shape(dist_actual)

In [ ]:
predictions1 = model.predict(X_test1)
predictions1 = predictions1-5
time = np.linspace(0,75,X_test1.shape[0])
plt.plot(time,predictions1,'o')
plt.plot(np.arange(75),dist_actual,'r')
plt.axis([0, 75, 0, 10],fontsize=20)

In [ ]:
max(predictions1)

In [ ]:
plt.imshow(X_test1[7,:,:,0])

In [ ]:
y_test